<div style="color:white;
           display:fill;
           border-radius:5px;
           background-color:#5642C5;
           font-size:200%;
           font-family:Arial;letter-spacing:0.5px">

<p width = 20%, style="padding: 10px;
              color:white;">
Pandas: Reshaping the data
              
</p>
</div>

Data Science Cohort Live NYC Feb 2022
<p>Phase 1: Topic 5</p>
<br>
<br>

<div align = "right">
<img src="Images/flatiron-school-logo.png" align = "right" width="200"/>
</div>
    
    

Data in the fires of the forge!


<figure><center><img src = "Images/conanbarbarian.png" width = 500></center>
</figure>



Transformation:
    
<figure><center><img src = "Images/conanthebarbarian5.png" width = 500></center>
</figure>

A suitably shaped dataset will make it easier to visualize/answer your questions.

#### Basic Ideas of Data Shaping in Pandas
1. Wide vs. Long Formats


<div>
    <center><img src="Images/hw_wide.png" align = "center" width="200"/></center>
    <center>Wide format</center>
</div>
    

<div align>
        <center><img src="Images/hw_long.png" align = "center" width="200"/></center>
    <center>Long format</center>
</div>

2. Multi-indexing (Hierarchical indexing)
- Saw this with multiple grouping.
- Very useful in quickly exploring responses conditioned on different factors.

<div>
    <center><img src="Images/hw_multi.png" align = "center" width="200"/></center> <br>
    <center>Multi-indexed Dataframe</center>
</div>
    

<div>
    <center><img src="Images/hw_multi_swap.png" align = "center" width="200"/></center>
    <br>
    <center>After index level swap</center>
</div>

#### Pivoting

- Convert from a long to a wide format:

   - DataFrame.pivot(index, columns, values):
  
 One attribute becomes index, values in other attribute becomes labels for new columns.
 
 Best to see an example:

In [1]:
import itertools
import pandas as pd
import numpy as np
value_list = [182, 160, 130, 78, 67, 52]
physical_data = pd.DataFrame(np.array([['John', 'Christopher', 'Melinda']*2, ['Height', 'Weight']*3, value_list]).T,
             columns = ['name', 'attribute', 'value'])

physical_data.head()

,name,attribute,value
0,John,Height,182
1,Christopher,Weight,160
2,Melinda,Height,130
3,John,Weight,78
4,Christopher,Height,67


This is long form. Use pivot to convert to wide format:

In [3]:
wide_form = physical_data.pivot(index = 'name', columns = 'attribute', values = 'value')
wide_form

attribute,Height,Weight
name,,
Christopher,67,160
John,182,78
Melinda,130,52


#### Melting: the inverse of pivoting.

- Take data from wide to long format.
- pd.melt(dataframe, id_vars, value_vars, var_name, value_name)

In [4]:
wide_form.reset_index(inplace = True)
wide_form

attribute,name,Height,Weight
0,Christopher,67,160
1,John,182,78
2,Melinda,130,52


In [6]:
pd.melt(wide_form, 
        id_vars = ['name'], 
        value_vars = ['Height', 'Weight'])

,name,attribute,value
0,Christopher,Height,67
1,John,Height,182
2,Melinda,Height,130
3,Christopher,Weight,160
4,John,Weight,78
5,Melinda,Weight,52


#### Pivot Tables

- When the columns you want to pivot on have non-unique entries.
- E.g., temperature as function of position X,Y for a given month but multiple measurements at each X,Y
- Want average of these measurements at each X,Y in pivoted form:

    - df.pivot_table(..., aggfunc = __)
    

Forest fire dataset:

Looks at temperature logged at various X, Y positions in a forest over several months.

In [7]:
forest_df = pd.read_csv('Data/forestfires.csv', usecols = ['X', 'Y', 'month', 'day', 'temp'])
inamonth_df = forest_df[(forest_df['month'] == 'mar')]

inamonth_df.head(10)

,X,Y,month,day,temp
0,7,5,mar,fri,8.2
3,8,6,mar,fri,8.3
4,8,6,mar,sun,11.4
16,5,5,mar,sat,15.1
18,6,4,mar,wed,15.9
39,4,4,mar,tue,14.1
48,4,4,mar,mon,11.8
49,4,4,mar,mon,11.0
60,2,2,mar,sun,11.5
61,2,2,mar,sun,5.5


Average temperature at (X, Y) positions for March. Organized as pivot table:

In [8]:
inamonth_df.pivot_table(index = 'X', columns = 'Y', values = 'temp', aggfunc = 'mean')

Y,2,3,4,5,6
X,,,,,
1,NaN,8.3,NaN,NaN,NaN
2,8.5,NaN,15.2000,NaN,NaN
3,NaN,NaN,13.6875,13.133333,NaN
4,NaN,11.0,14.1000,15.975000,12.30
5,NaN,NaN,14.4000,13.350000,15.10
6,NaN,15.2,14.6000,12.260000,NaN
7,NaN,11.0,13.8000,8.200000,NaN
8,NaN,NaN,NaN,NaN,12.15


#### Multiindexing
- Setting multiple columns as index
- Setting hierarchies.
- Accessing data in multi-indexed DataFrames.

Airfoil noise dataset:
- Various factors affecting sound amplitude off of airplane wings.

In [9]:
colnames = ['Frequency [Hz]', 'Angle of attack [deg]', 'Chord length [m]', 'Free-stream velocity [m/s]', 'Suction side thickness [m]', 'Sound volume [dB]']
airfoil_df = pd.read_csv('Data/airfoil_self_noise.dat', delimiter='\t', header = None, names = colnames  )

airfoil_df.head()

,Frequency [Hz],Angle of attack [deg],Chord length [m],Free-stream velocity [m/s],Suction side thickness [m],Sound volume [dB]
0,800,0.0,0.3048,71.3,0.002663,126.201
1,1000,0.0,0.3048,71.3,0.002663,125.201
2,1250,0.0,0.3048,71.3,0.002663,125.951
3,1600,0.0,0.3048,71.3,0.002663,127.591
4,2000,0.0,0.3048,71.3,0.002663,127.461


Setting multiple attributes as indices can give us flexibility in addressing the data.
- How does sound amplitude depend just on frequency, stream velocity, and foil chord length?
- Create hierarchical Multiindex:

In [12]:
col_subset = ['Frequency [Hz]', 'Free-stream velocity [m/s]', 'Angle of attack [deg]', 'Sound volume [dB]']
airfoil_df = airfoil_df[col_subset].set_index(col_subset[0:3])
airfoil_df.head()

,,,Sound volume [dB]
Frequency [Hz],Free-stream velocity [m/s],Angle of attack [deg],
800,71.3,0.0,126.201
1000,71.3,0.0,125.201
1250,71.3,0.0,125.951
1600,71.3,0.0,127.591
2000,71.3,0.0,127.461


Moved columns to index, but hierarchical structure of indices not set:
- Can be accomplished with the .sort_index() method.

In [16]:
airfoil_df = airfoil_df.sort_index()
airfoil_df

Sound volume [dB]
Frequency [Hz] Free-stream velocity [m/s] Angle of attack [deg]                   
200            31.7                       0.0                              117.195
                                          7.3                              128.679
                                          9.5                              119.146
                                          9.9                              127.299
                                          12.3                             124.987
...                                                                            ...
20000          55.5                       4.8                              121.284
               71.3                       0.0                              114.474
                                          0.0                              123.217
                                          4.0                              109.638
                                          4.8                              119.708

[1503 rows x 1 columns]

#### Accessing via the .loc accessor on multi-indices
-DataFrame.loc[first_level_index, columns]
- Dataframe.loc[(first_level_index, second_level_index, third_level_index), columns]

In [11]:
# at frequency = 1000 Hz
airfoil_df.loc[1000, :]

Sound volume [dB]
Free-stream velocity [m/s] Angle of attack [deg]                   
31.7                       0.0                              127.365
                           0.0                              130.595
                           0.0                              130.987
                           0.0                              126.780
                           0.0                              125.127
...                                                             ...
71.3                       15.4                             131.865
                           15.6                             124.438
                           17.4                             125.816
                           19.7                             121.885
                           22.2                             134.247

[99 rows x 1 columns]

In [12]:
# sound vol vs angle of attack
# fixed at 1000 Hz, 55.5 m/s stream velocity
airfoil_df.loc[(1000, 55.5)]

,Sound volume [dB]
Angle of attack [deg],
0.0,126.966
0.0,128.290
0.0,128.583
0.0,125.586
0.0,129.800
2.0,129.002
3.0,129.329
3.3,131.129
4.0,132.134


Swapping level hierarchy:
- Look at measurement/response keeping one variable fixed and varying another.
- Swapping level hierarchy switches which we keep fixed and which we vary.


In [17]:
swapped_df = airfoil_df.swaplevel('Free-stream velocity [m/s]', 'Angle of attack [deg]').sort_index()

In [18]:
swapped_df.head()

Sound volume [dB]
Frequency [Hz] Angle of attack [deg] Free-stream velocity [m/s]                   
200            0.0                   31.7                                  117.195
                                     39.6                                  118.129
               7.3                   31.7                                  128.679
                                     39.6                                  130.989
                                     55.5                                  135.234

In [15]:

swapped_df.loc[(1000, 7.3)]

,Sound volume [dB]
Free-stream velocity [m/s],
31.7,119.099
39.6,122.539
55.5,125.194
71.3,126.838


At fixed angle of attack, sound volume increases with airflow speed at 1 kHz.

Multi-indexing opens up many possibilities for data manipulation.

Strongly encourage you to look at supplementary material and pandas documentation!